# Executive Summary

This project helps Target Corporation make data-driven retail expansion decisions across the United States.
By combining store location data, U.S. Census demographics, and income statistics, the analysis identifies counties with no Target stores but high population and household income — areas representing strong expansion potential.

The final Tableau dashboard visualizes:

Store coverage across U.S. counties

Demographic patterns (population & income)

Top recommended counties for new Target stores

🧭 Objectives

Analyze Target’s current geographic store distribution.

Combine demographic indicators (population, income) at the county level.

Detect underserved counties with above-average population and income.

Generate actionable expansion insights for Target’s strategy team.

⚙️ Analysis Workflow

Data Cleaning

   Standardized county and state names

   Built County_State_Key for consistent merging

   Normalized formatting, removed suffixes like “County” or “Parish”

   Merging & Aggregation

   Aggregated store counts by county

   Merged population and income data

   Created unified file county_level_summary.csv

Opportunity Identification

   Calculated national averages:

   Population ≈ 105,140

   Median HH Income ≈ $65,104

   Filtered counties with:

   No Target stores

   Above-average population & income

   Result: 60 high-potential counties for expansion

State-Level Summary

   Aggregated opportunity counts by state

   Top states for expansion

In [45]:
import pandas as pd 
import numpy as np 

Cleaning and dropping of columns from income and population datasets to keep only the needed ones.

In [67]:
# File paths
store_fp = "Target datasets.csv"
pop_fp   = "county_population_2023.csv"
inc_fp   = "median_income_by_county_2023.csv"

# Load data (population/income have a header row to skip)
df_stores     = pd.read_csv(store_fp, encoding="ISO-8859-1", low_memory=False)
df_population = pd.read_csv(pop_fp, skiprows=1, dtype=str, low_memory=False)
df_income     = pd.read_csv(inc_fp, skiprows=1, dtype=str, low_memory=False)

print("✅ Loaded datasets")
print("Stores shape:", df_stores.shape)
print("Population shape:", df_population.shape)
print("Income shape:", df_income.shape)


✅ Loaded datasets
Stores shape: (1780, 47)
Population shape: (3224, 379)
Income shape: (3224, 243)


# Cleaning


PURPOSE:
Define reusable cleaning functions for county/state names and key creation.

In [68]:


import re

# map state abbreviations to full names
US_STATE_MAP = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio',
    'OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina',
    'SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia',
    'WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}

# regex pattern to strip suffixes like County, Parish, Borough, etc.
_SUFFIX_RE = re.compile(r'\b(County|Parish|Borough|Municipality|Census Area|City|Municipio)\b', re.IGNORECASE)

def clean_county_name(name: str):
    """Remove county-type suffixes, trim spaces, and title-case."""
    if pd.isna(name):
        return None
    name = str(name)
    name = _SUFFIX_RE.sub('', name)
    name = name.strip()
    return name.title() if name else None

def standardize_state_name(state: str):
    """Convert 2-letter state code to full name if possible, else title-case."""
    if pd.isna(state):
        return None
    s = str(state).strip()
    if len(s) == 2 and s.upper() in US_STATE_MAP:
        return US_STATE_MAP[s.upper()]
    return s.title()

def make_key(county: str, state: str):
    """Build 'County|State' key if both exist."""
    if not county or not state:
        return None
    return f"{county}|{state}"


PURPOSE:
 Clean county & state in df_stores and create a stable key: County|State.
 (Uses the helper functions from step 1.1)


In [ ]:

# 1) filter to United States if the column exists
if "Address.CountryName" in df_stores.columns:
    df_stores = df_stores[
        df_stores["Address.CountryName"].astype(str).str.contains("United States", case=False, na=False)
    ].copy()



Null County_Clean: 1
Null State_Clean : 0
Null Key         : 1


,County_Clean,State_Clean,County_State_Key
0,Tuscaloosa,Alabama,Tuscaloosa|Alabama
1,Houston,Georgia,Houston|Georgia
2,Barrow,Georgia,Barrow|Georgia
3,Hall,Georgia,Hall|Georgia
4,Madison,Alabama,Madison|Alabama
5,Morgan,Alabama,Morgan|Alabama
6,Hall,Georgia,Hall|Georgia
7,Houston,Alabama,Houston|Alabama
8,Madison,Alabama,Madison|Alabama
9,Dougherty,Georgia,Dougherty|Georgia


✅ Saved stores_clean.csv with 1779 rows


In [ ]:
# 2) pick the original columns for county & state from your schema
county_col = "Address.County"
state_col  = "Address.Subdivision"   # usually a 2-letter code like CA, TX, etc.




In [ ]:
# 3) apply cleaning helpers
df_stores["County_Clean"] = df_stores[county_col].apply(clean_county_name)
df_stores["State_Clean"]  = df_stores[state_col].apply(standardize_state_name)
df_stores["County_State_Key"] = df_stores.apply(
    lambda r: make_key(r["County_Clean"], r["State_Clean"]), axis=1
)



In [ ]:
# 4) quick sanity check
print("Null County_Clean:", df_stores["County_Clean"].isna().sum())
print("Null State_Clean :", df_stores["State_Clean"].isna().sum())
print("Null Key         :", df_stores["County_State_Key"].isna().sum())

# 5) preview
display(df_stores[["County_Clean","State_Clean","County_State_Key"]].head(10))



In [ ]:
# 6) save a cleaned version for later steps
stores_clean = df_stores[df_stores["County_State_Key"].notna()].copy()
stores_clean.to_csv("stores_clean.csv", index=False)
print(f"✅ Saved stores_clean.csv with {stores_clean.shape[0]} rows")

PURPOSE:
 Parse county & state from "Geographic Area Name", clean them, make a key,
 extract numeric total population, and save an interim clean file.

In [ ]:

# 1) identify columns
geo_col = "Geographic Area Name"
pop_col_candidates = [c for c in df_population.columns
                      if c.startswith("Estimate") and "Total population" in c]
if not pop_col_candidates:
    raise KeyError("Could not find the Total population estimate column in df_population.")
pop_total_col = pop_col_candidates[0]



Null County_Clean: 0
Null State_Clean : 2
Null Key         : 2
Null Population  : 0


,Geographic Area Name,County_Clean,State_Clean,County_State_Key,Population_2023
0,United States,United States,None,None,332387540.0
1,United States -- Rural,United States -- Rural,None,None,66686736.0
2,"Autauga County, Alabama",Autauga,Alabama,Autauga|Alabama,59285.0
3,"Baldwin County, Alabama",Baldwin,Alabama,Baldwin|Alabama,239945.0
4,"Barbour County, Alabama",Barbour,Alabama,Barbour|Alabama,24757.0
5,"Bibb County, Alabama",Bibb,Alabama,Bibb|Alabama,22152.0
6,"Blount County, Alabama",Blount,Alabama,Blount|Alabama,59292.0
7,"Bullock County, Alabama",Bullock,Alabama,Bullock|Alabama,10157.0
8,"Butler County, Alabama",Butler,Alabama,Butler|Alabama,18807.0
9,"Calhoun County, Alabama",Calhoun,Alabama,Calhoun|Alabama,116141.0


✅ Saved county_population_clean.csv with 3222 rows


In [ ]:
# 2) split "Geographic Area Name" -> county + state
def split_geo(geo):
    if pd.isna(geo):
        return pd.Series([None, None])
    parts = [p.strip() for p in str(geo).split(",")]
    county_raw = parts[0] if parts else None
    state_raw  = parts[-1] if len(parts) >= 2 else None
    return pd.Series([clean_county_name(county_raw),
                      standardize_state_name(state_raw)])

pop_clean = df_population.copy()
pop_clean[["County_Clean","State_Clean"]] = pop_clean[geo_col].apply(split_geo)



In [ ]:
# 3) make key
pop_clean["County_State_Key"] = pop_clean.apply(
    lambda r: make_key(r["County_Clean"], r["State_Clean"]), axis=1
)

# 4) numeric total population
pop_clean["Population_2023"] = (
    pop_clean[pop_total_col]
      .astype(str)
      .str.replace(",", "", regex=False)
      .str.extract(r"(\d+)", expand=False)
      .astype(float)
)



In [ ]:
# 5) keep only needed columns
pop_clean = pop_clean[[geo_col, "County_Clean", "State_Clean",
                       "County_State_Key", "Population_2023"]].copy()

# 6) sanity checks + preview
print("Null County_Clean:", pop_clean["County_Clean"].isna().sum())
print("Null State_Clean :", pop_clean["State_Clean"].isna().sum())
print("Null Key         :", pop_clean["County_State_Key"].isna().sum())
print("Null Population  :", pop_clean["Population_2023"].isna().sum())

display(pop_clean.head(10))

# 7) save interim
pop_clean_valid = pop_clean[pop_clean["County_State_Key"].notna()].copy()
pop_clean_valid.to_csv("county_population_clean.csv", index=False)
print(f"✅ Saved county_population_clean.csv with {pop_clean_valid.shape[0]} rows")


PURPOSE: Locate the exact "Median household income" estimate column in df_income.

In [91]:
# show all median income column names so we can find the overall one
for i, c in enumerate([c for c in df_income.columns if "median" in c.lower() and "income" in c.lower()], 1):
    print(f"{i}. {c}")


1. Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households
2. Margin of Error!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households
3. Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!White
4. Margin of Error!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!White
5. Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Black or African American
6. Margin of Error!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!Black or African American
7. Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households!!One race--!!American I

PURPOSE:
 Clean df_income using the general median household income column.

In [ ]:
inc_geo_col = "Geographic Area Name"
inc_median_col = "Estimate!!Median income (dollars)!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households"

# split "Geographic Area Name" -> county + state
def _split_geo_income(geo):
    if pd.isna(geo):
        return pd.Series([None, None])
    parts = [p.strip() for p in str(geo).split(",")]
    county_raw = parts[0] if parts else None
    state_raw  = parts[-1] if len(parts) >= 2 else None
    return pd.Series([clean_county_name(county_raw),
                      standardize_state_name(state_raw)])

inc_clean = df_income.copy()
inc_clean[["County_Clean","State_Clean"]] = inc_clean[inc_geo_col].apply(_split_geo_income)

# make key
inc_clean["County_State_Key"] = inc_clean.apply(
    lambda r: make_key(r["County_Clean"], r["State_Clean"]), axis=1
)

# numeric median income (remove $, commas)
inc_clean["Median_HH_Income_2023"] = (
    inc_clean[inc_median_col]
      .astype(str)
      .str.replace("$", "", regex=False)
      .str.replace(",", "", regex=False)
      .str.extract(r"(\d+\.?\d*)", expand=False)
      .astype(float)
)

# keep key columns
inc_clean = inc_clean[[inc_geo_col, "County_Clean", "State_Clean",
                       "County_State_Key", "Median_HH_Income_2023"]].copy()

# check + preview
print("Null County_Clean:", inc_clean["County_Clean"].isna().sum())
print("Null State_Clean :", inc_clean["State_Clean"].isna().sum())
print("Null Key         :", inc_clean["County_State_Key"].isna().sum())
print("Null MedianInc   :", inc_clean["Median_HH_Income_2023"].isna().sum())
display(inc_clean.head(10))

# save interim
inc_clean_valid = inc_clean[inc_clean["County_State_Key"].notna()].copy()
inc_clean_valid.to_csv("county_income_clean.csv", index=False)
print(f"✅ Saved county_income_clean.csv with {inc_clean_valid.shape[0]} rows")


Null County_Clean: 0
Null State_Clean : 2
Null Key         : 2
Null MedianInc   : 2


,Geographic Area Name,County_Clean,State_Clean,County_State_Key,Median_HH_Income_2023
0,United States,United States,None,None,78538.0
1,United States -- Rural,United States -- Rural,None,None,74427.0
2,"Autauga County, Alabama",Autauga,Alabama,Autauga|Alabama,69841.0
3,"Baldwin County, Alabama",Baldwin,Alabama,Baldwin|Alabama,75019.0
4,"Barbour County, Alabama",Barbour,Alabama,Barbour|Alabama,44290.0
5,"Bibb County, Alabama",Bibb,Alabama,Bibb|Alabama,51215.0
6,"Blount County, Alabama",Blount,Alabama,Blount|Alabama,61096.0
7,"Bullock County, Alabama",Bullock,Alabama,Bullock|Alabama,36723.0
8,"Butler County, Alabama",Butler,Alabama,Butler|Alabama,44881.0
9,"Calhoun County, Alabama",Calhoun,Alabama,Calhoun|Alabama,55826.0


✅ Saved county_income_clean.csv with 3222 rows


PURPOSE:
 Aggregate the cleaned Target stores dataset to the county level
 to calculate how many stores exist in each county.

In [75]:

# 2) group by County_State_Key
store_counts = (
    stores_clean.groupby("County_State_Key", dropna=True)
    .agg(
        Store_Count=("County_State_Key", "size"),
        County=("County_Clean", "first"),
        State=("State_Clean", "first")
    )
    .reset_index(drop=True)
)

# 3) sort nicely
store_counts = store_counts.sort_values(["State", "County"]).reset_index(drop=True)

# 4) preview
print("✅ Store counts created:")
display(store_counts.head(10))

# 5) save for later joins
store_counts.to_csv("county_store_counts.csv", index=False)
print(f"✅ Saved county_store_counts.csv with {store_counts.shape[0]} rows")


✅ Store counts created:


,Store_Count,County,State
0,2,Baldwin,Alabama
1,1,Houston,Alabama
2,1,Lauderdale,Alabama
3,3,Madison,Alabama
4,2,Mobile,Alabama
5,1,Morgan,Alabama
6,1,Tuscaloosa,Alabama
7,2,Anchorage,Alaska
8,1,Matanuska-Susitna,Alaska
9,1,Cochise,Arizona


✅ Saved county_store_counts.csv with 619 rows


PURPOSE:
 Rebuild county_store_counts so it includes County_State_Key (needed for joining).

In [ ]:
stores_clean = pd.read_csv("stores_clean.csv")

store_counts = (
    stores_clean
      .groupby(["County_State_Key", "County_Clean", "State_Clean"], dropna=True)
      .size()
      .reset_index(name="Store_Count")
      .rename(columns={"County_Clean":"County", "State_Clean":"State"})
)

display(store_counts.head(10))
store_counts.to_csv("county_store_counts.csv", index=False)
print(f"✅ Re-saved county_store_counts.csv with {store_counts.shape[0]} rows and County_State_Key")


,County_State_Key,County,State,Store_Count
0,Adams|Colorado,Adams,Colorado,3
1,Ada|Idaho,Ada,Idaho,2
2,Aiken|South Carolina,Aiken,South Carolina,1
3,Alachua|Florida,Alachua,Florida,2
4,Alamance|North Carolina,Alamance,North Carolina,1
5,Alameda|California,Alameda,California,13
6,Albany|New York,Albany,New York,2
7,Albemarle|Virginia,Albemarle,Virginia,1
8,Alexandria|Virginia,Alexandria,Virginia,1
9,Allegheny|Pennsylvania,Allegheny,Pennsylvania,11


✅ Re-saved county_store_counts.csv with 619 rows and County_State_Key


# MERGING

PURPOSE:
 Merge population, income, and store counts into one master dataset.

In [78]:

pop_clean = pd.read_csv("county_population_clean.csv")[
    ["County_State_Key", "County_Clean", "State_Clean", "Population_2023"]
]
inc_clean = pd.read_csv("county_income_clean.csv")[
    ["County_State_Key", "Median_HH_Income_2023"]
]
store_counts = pd.read_csv("county_store_counts.csv")[
    ["County_State_Key", "Store_Count"]
]

county_summary = (
    pop_clean
      .merge(inc_clean, on="County_State_Key", how="left")
      .merge(store_counts, on="County_State_Key", how="left")
)

county_summary["Store_Count"] = county_summary["Store_Count"].fillna(0).astype(int)

# Tidy columns
county_summary = county_summary.rename(columns={
    "County_Clean":"County",
    "State_Clean":"State"
})[["County_State_Key","County","State","Population_2023","Median_HH_Income_2023","Store_Count"]]

display(county_summary.head(10))
print("✅ Merged dataset shape:", county_summary.shape)

county_summary.to_csv("county_level_summary.csv", index=False)
print("✅ Saved county_level_summary.csv")


,County_State_Key,County,State,Population_2023,Median_HH_Income_2023,Store_Count
0,Autauga|Alabama,Autauga,Alabama,59285.0,69841.0,0
1,Baldwin|Alabama,Baldwin,Alabama,239945.0,75019.0,2
2,Barbour|Alabama,Barbour,Alabama,24757.0,44290.0,0
3,Bibb|Alabama,Bibb,Alabama,22152.0,51215.0,0
4,Blount|Alabama,Blount,Alabama,59292.0,61096.0,0
5,Bullock|Alabama,Bullock,Alabama,10157.0,36723.0,0
6,Butler|Alabama,Butler,Alabama,18807.0,44881.0,0
7,Calhoun|Alabama,Calhoun,Alabama,116141.0,55826.0,0
8,Chambers|Alabama,Chambers,Alabama,34450.0,49295.0,0
9,Cherokee|Alabama,Cherokee,Alabama,25224.0,50769.0,0


✅ Merged dataset shape: (3234, 6)
✅ Saved county_level_summary.csv


PURPOSE:
 Validate that Store_Count values in county_store_counts.csv look correct.

In [79]:


store_counts = pd.read_csv("county_store_counts.csv")

# 1) quick preview
print("🔹 Preview of store counts:")
display(store_counts.sample(10, random_state=1))

# 2) summary stats
print("\n🔹 Basic stats:")
print(store_counts["Store_Count"].describe())

# 3) manual check — pick a known county from your cleaned store list
stores_clean = pd.read_csv("stores_clean.csv")

example_key = "Cook|Illinois"   # 👈 change this to any County|State you want to verify
manual_count = stores_clean[stores_clean["County_State_Key"] == example_key].shape[0]

print(f"\n🔹 Manual check for {example_key}: {manual_count} stores in cleaned store data")

# 4) confirm consistency
auto_count = store_counts.loc[store_counts["County_State_Key"] == example_key, "Store_Count"]
if not auto_count.empty:
    print(f"🔹 Aggregated count in store_counts: {int(auto_count.values[0])}")
else:
    print("⚠️ That county not found in store_counts.csv — check spelling or format.")


🔹 Preview of store counts:


,County_State_Key,County,State,Store_Count
557,Ventura|California,Ventura,California,8
195,Frederick|Maryland,Frederick,Maryland,1
200,Garfield|Colorado,Garfield,Colorado,1
394,Norfolk|Virginia,Norfolk,Virginia,1
23,Bay|Florida,Bay,Florida,2
47,Bowie|Texas,Bowie,Texas,1
284,Kings|New York,Kings,New York,5
406,Ontario|New York,Ontario,New York,1
422,Penobscot|Maine,Penobscot,Maine,1
536,Taney|Missouri,Taney,Missouri,1



🔹 Basic stats:
count    619.00000
mean       2.87399
std        4.65269
min        1.00000
25%        1.00000
50%        1.00000
75%        3.00000
max       69.00000
Name: Store_Count, dtype: float64

🔹 Manual check for Cook|Illinois: 44 stores in cleaned store data
🔹 Aggregated count in store_counts: 44


PURPOSE:
 Create a single county-level table with Population, Median Income, and Store_Count.

In [80]:


import pandas as pd

# 1) load the cleaned/intermediate files
pop_clean    = pd.read_csv("county_population_clean.csv")[
    ["County_State_Key", "County_Clean", "State_Clean", "Population_2023"]
]
inc_clean    = pd.read_csv("county_income_clean.csv")[
    ["County_State_Key", "Median_HH_Income_2023"]
]
store_counts = pd.read_csv("county_store_counts.csv")[
    ["County_State_Key", "Store_Count"]
]

# 2) merge population + income
county_summary = pop_clean.merge(inc_clean, on="County_State_Key", how="left")

# 3) merge store counts (left-join so counties without stores remain)
county_summary = county_summary.merge(store_counts, on="County_State_Key", how="left")

# 4) tidy up
county_summary["Store_Count"] = county_summary["Store_Count"].fillna(0).astype(int)
county_summary = county_summary.rename(columns={
    "County_Clean":"County",
    "State_Clean":"State"
})[["County_State_Key","County","State","Population_2023","Median_HH_Income_2023","Store_Count"]]

# 5) quick preview + save
display(county_summary.head(10))
print("✅ Merged dataset shape:", county_summary.shape)

county_summary.to_csv("county_level_summary.csv", index=False)
print("✅ Saved county_level_summary.csv")


,County_State_Key,County,State,Population_2023,Median_HH_Income_2023,Store_Count
0,Autauga|Alabama,Autauga,Alabama,59285.0,69841.0,0
1,Baldwin|Alabama,Baldwin,Alabama,239945.0,75019.0,2
2,Barbour|Alabama,Barbour,Alabama,24757.0,44290.0,0
3,Bibb|Alabama,Bibb,Alabama,22152.0,51215.0,0
4,Blount|Alabama,Blount,Alabama,59292.0,61096.0,0
5,Bullock|Alabama,Bullock,Alabama,10157.0,36723.0,0
6,Butler|Alabama,Butler,Alabama,18807.0,44881.0,0
7,Calhoun|Alabama,Calhoun,Alabama,116141.0,55826.0,0
8,Chambers|Alabama,Chambers,Alabama,34450.0,49295.0,0
9,Cherokee|Alabama,Cherokee,Alabama,25224.0,50769.0,0


✅ Merged dataset shape: (3234, 6)
✅ Saved county_level_summary.csv


PURPOSE:
 Verify that Store_Count values in county_summary match the original store_counts data.

In [81]:

# Load both
county_summary = pd.read_csv("county_level_summary.csv")
store_counts   = pd.read_csv("county_store_counts.csv")

# 1) Preview Store_Count distribution
print("🔹 Basic stats for merged Store_Count column:")
print(county_summary["Store_Count"].describe())

# 2) Check a few random samples from merged file
print("\n🔹 Random sample of merged Store_Counts:")
display(county_summary.sample(10, random_state=42))

# 3) Validate consistency — compare totals between both sources
merged_total = county_summary["Store_Count"].sum()
original_total = store_counts["Store_Count"].sum()
print(f"\n🔹 Total stores (original): {original_total}")
print(f"🔹 Total stores (merged):   {merged_total}")

# 4) Spot-check for mismatch — any counties where counts differ
merged_check = county_summary.merge(
    store_counts, on="County_State_Key", how="inner", suffixes=("_merged", "_original")
)
mismatch = merged_check[
    merged_check["Store_Count_merged"] != merged_check["Store_Count_original"]
]

if mismatch.empty:
    print("\n✅ Store counts match perfectly after merge.")
else:
    print(f"\n⚠️ Found {len(mismatch)} mismatched counties — showing first 10:")
    display(mismatch.head(10))


🔹 Basic stats for merged Store_Count column:
count    3234.000000
mean        0.569573
std         2.365912
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        69.000000
Name: Store_Count, dtype: float64

🔹 Random sample of merged Store_Counts:


,County_State_Key,County,State,Population_2023,Median_HH_Income_2023,Store_Count
1334,Cottonwood|Minnesota,Cottonwood,Minnesota,11458.0,68239.0,0
1756,Elko|Nevada,Elko,Nevada,53852.0,83427.0,0
3031,Marshall|West Virginia,Marshall,West Virginia,30129.0,60329.0,0
321,District Of Columbia|District Of Columbia,District Of Columbia,District Of Columbia,672079.0,106287.0,0
2517,Tipton|Tennessee,Tipton,Tennessee,61339.0,71736.0,0
1251,Chippewa|Michigan,Chippewa,Michigan,36448.0,60631.0,0
1192,Waldo|Maine,Waldo,Maine,40006.0,68441.0,0
1206,Garrett|Maryland,Garrett,Maryland,28713.0,69031.0,0
1309,St. Joseph|Michigan,St. Joseph,Michigan,60870.0,64707.0,0
997,Anderson|Kentucky,Anderson,Kentucky,24098.0,71747.0,0



🔹 Total stores (original): 1779
🔹 Total stores (merged):   1842

✅ Store counts match perfectly after merge.


PURPOSE:
 Check whether County_State_Key values match between population and store_counts.
 If there is no overlap, the merge will produce all zeros.


In [82]:

pop_clean    = pd.read_csv("county_population_clean.csv")[["County_State_Key","County_Clean","State_Clean"]]
store_counts = pd.read_csv("county_store_counts.csv")[["County_State_Key","County","State","Store_Count"]]

pop_keys   = set(pop_clean["County_State_Key"].dropna().unique())
store_keys = set(store_counts["County_State_Key"].dropna().unique())

print("Unique keys — population:", len(pop_keys))
print("Unique keys — stores    :", len(store_keys))
print("Exact overlap            :", len(pop_keys & store_keys))

# Show a few examples that don't match (stores not in population)
stores_not_in_pop = sorted(store_keys - pop_keys)
print("\nExamples in stores but NOT in population (first 15):")
for k in stores_not_in_pop[:15]:
    print("  ", k)

# And a few examples the other way (population not in stores)
pop_not_in_stores = sorted(pop_keys - store_keys)
print("\nExamples in population but NOT in stores (first 15):")
for k in pop_not_in_stores[:15]:
    print("  ", k)


Unique keys — population: 3216
Unique keys — stores    : 619
Exact overlap            : 609

Examples in stores but NOT in population (first 15):
   District Of Columbia|District of Columbia
   Dona Ana|New Mexico
   Fairfield|Connecticut
   Hartford|Connecticut
   La Salle|Illinois
   Litchfield|Connecticut
   New Haven|Connecticut
   New London|Connecticut
   Pa - Pennsylvania|Pennsylvania
   Windham|Connecticut

Examples in population but NOT in stores (first 15):
   Abbeville|South Carolina
   Acadia|Louisiana
   Accomack|Virginia
   Adair|Iowa
   Adair|Kentucky
   Adair|Missouri
   Adair|Oklahoma
   Adams|Idaho
   Adams|Illinois
   Adams|Indiana
   Adams|Iowa
   Adams|Mississippi
   Adams|Nebraska
   Adams|North Dakota
   Adams|Ohio


PURPOSE:
 Normalize State casing (Title Case), remove accents from County/State,
 rebuild County_State_Key consistently across ALL three cleaned files.

In [85]:
pip install unidecode

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [86]:
from unidecode import unidecode  

def normalize_cols(df, county_col="County_Clean", state_col="State_Clean"):
    df[county_col] = df[county_col].astype(str).map(lambda x: unidecode(x).strip() if x not in [None, "nan"] else None)
    df[state_col]  = df[state_col].astype(str).map(lambda x: unidecode(x).strip() if x not in [None, "nan"] else None)
    # enforce Title Case on both to kill 'of' vs 'Of' differences
    df[county_col] = df[county_col].map(lambda x: x.title() if isinstance(x, str) and x else x)
    df[state_col]  = df[state_col].map(lambda x: x.title() if isinstance(x, str) and x else x)
    df["County_State_Key"] = df.apply(lambda r: f"{r[county_col]}|{r[state_col]}" if pd.notna(r[county_col]) and pd.notna(r[state_col]) else None, axis=1)
    return df

# 1) normalize the three cleaned files
stores_clean = pd.read_csv("stores_clean.csv")
pop_clean    = pd.read_csv("county_population_clean.csv")
inc_clean    = pd.read_csv("county_income_clean.csv")

stores_clean = normalize_cols(stores_clean, "County_Clean", "State_Clean")
pop_clean    = normalize_cols(pop_clean,    "County_Clean", "State_Clean")
inc_clean    = normalize_cols(inc_clean,    "County_Clean", "State_Clean")

# 2) re-save
stores_clean.to_csv("stores_clean.csv", index=False)
pop_clean.to_csv("county_population_clean.csv", index=False)
inc_clean.to_csv("county_income_clean.csv", index=False)

print("✅ Re-saved normalized cleaned files with consistent keys.")


✅ Re-saved normalized cleaned files with consistent keys.


 PURPOSE: rebuild store_counts with normalized keys, then re-merge master.

In [87]:

stores_clean = pd.read_csv("stores_clean.csv")
store_counts = (
    stores_clean
      .groupby(["County_State_Key", "County_Clean", "State_Clean"], dropna=True)
      .size()
      .reset_index(name="Store_Count")
      .rename(columns={"County_Clean":"County", "State_Clean":"State"})
)
store_counts.to_csv("county_store_counts.csv", index=False)

# re-merge master
pop_clean = pd.read_csv("county_population_clean.csv")[["County_State_Key","County_Clean","State_Clean","Population_2023"]]
inc_clean = pd.read_csv("county_income_clean.csv")[["County_State_Key","Median_HH_Income_2023"]]

county_summary = (
    pop_clean
      .merge(inc_clean, on="County_State_Key", how="left")
      .merge(store_counts[["County_State_Key","Store_Count"]], on="County_State_Key", how="left")
)

county_summary["Store_Count"] = county_summary["Store_Count"].fillna(0).astype(int)
county_summary = county_summary.rename(columns={"County_Clean":"County","State_Clean":"State"})[
    ["County_State_Key","County","State","Population_2023","Median_HH_Income_2023","Store_Count"]
]
county_summary.to_csv("county_level_summary.csv", index=False)

print("✅ Rebuilt county_store_counts.csv and county_level_summary.csv")
display(county_summary.sample(10, random_state=2))


✅ Rebuilt county_store_counts.csv and county_level_summary.csv


,County_State_Key,County,State,Population_2023,Median_HH_Income_2023,Store_Count
2527,Williamson|Tennessee,Williamson,Tennessee,254609.0,131202.0,2
103,La Paz|Arizona,La Paz,Arizona,16605.0,49506.0,0
75,Dillingham|Alaska,Dillingham,Alaska,4780.0,74250.0,0
3027,Lincoln|West Virginia,Lincoln,West Virginia,20170.0,48593.0,0
1711,Keya Paha|Nebraska,Keya Paha,Nebraska,939.0,60313.0,0
322,Alachua|Florida,Alachua,Florida,281751.0,59659.0,2
1981,Surry|North Carolina,Surry,North Carolina,71407.0,56095.0,0
2719,Randall|Texas,Randall,Texas,143791.0,80905.0,0
1124,Calcasieu|Louisiana,Calcasieu,Louisiana,208668.0,67849.0,1
592,Shoshone|Idaho,Shoshone,Idaho,13580.0,49975.0,0


In [88]:

county_summary = pd.read_csv("county_level_summary.csv")
has_stores = county_summary[county_summary["Store_Count"] > 0]
print("Counties with stores:", has_stores.shape[0])
print("Total stores (sum):  ", has_stores["Store_Count"].sum())

# show 10 non-zero examples
display(has_stores.sample(10, random_state=7))


Counties with stores: 626
Total stores (sum):   1844


,County_State_Key,County,State,Population_2023,Median_HH_Income_2023,Store_Count
2932,Danville|Virginia,Danville,Virginia,42239.0,42778.0,1
678,St. Clair|Illinois,St. Clair,Illinois,254777.0,70178.0,3
1122,Bossier|Louisiana,Bossier,Louisiana,129134.0,66336.0,1
296,Pueblo|Colorado,Pueblo,Colorado,168726.0,62250.0,1
2919,Warren|Virginia,Warren,Virginia,41104.0,84331.0,1
1331,Clay|Minnesota,Clay,Minnesota,65628.0,77664.0,1
2527,Williamson|Tennessee,Williamson,Tennessee,254609.0,131202.0,2
451,Glynn|Georgia,Glynn,Georgia,84987.0,68546.0,1
1497,Buchanan|Missouri,Buchanan,Missouri,83855.0,59423.0,1
2755,Travis|Texas,Travis,Texas,1307625.0,97169.0,11


PURPOSE:
 Identify high-potential counties (no Target store + above-average pop & income)

In [89]:

county_summary = pd.read_csv("county_level_summary.csv")

# 1) compute national averages (ignore missing values)
avg_pop = county_summary["Population_2023"].mean(skipna=True)
avg_inc = county_summary["Median_HH_Income_2023"].mean(skipna=True)

print(f"📊 National average population: {avg_pop:,.0f}")
print(f"📊 National average income:     ${avg_inc:,.0f}")

# 2) flag opportunity counties
opportunities = county_summary[
    (county_summary["Store_Count"] == 0) &
    (county_summary["Population_2023"] >= avg_pop) &
    (county_summary["Median_HH_Income_2023"] >= avg_inc)
].copy()

# 3) sort by population descending
opportunities = opportunities.sort_values("Population_2023", ascending=False)

# 4) preview & save
print(f"\n✅ Found {opportunities.shape[0]} high-potential counties (no stores, above-average pop & income)")
display(opportunities.head(15))

opportunities.to_csv("target_expansion_opportunities.csv", index=False)
print("✅ Saved target_expansion_opportunities.csv")


📊 National average population: 105,140
📊 National average income:     $65,104

✅ Found 60 high-potential counties (no stores, above-average pop & income)


,County_State_Key,County,State,Population_2023,Median_HH_Income_2023,Store_Count
448,Fulton|Georgia,Fulton,Georgia,1068507.0,91490.0,0
309,Capitol Planning Region|Connecticut,Capitol Planning Region,Connecticut,969029.0,91541.0,0
455,Gwinnett|Georgia,Gwinnett,Georgia,966972.0,84823.0,0
421,Cobb|Georgia,Cobb,Georgia,769152.0,98712.0,0
432,Dekalb|Georgia,Dekalb,Georgia,762105.0,77683.0,0
317,Western Connecticut Planning Region|Connecticut,Western Connecticut Planning Region,Connecticut,621232.0,124553.0,0
315,South Central Connecticut Planning Region|Conn...,South Central Connecticut Planning Region,Connecticut,566803.0,86266.0,0
312,Naugatuck Valley Planning Region|Connecticut,Naugatuck Valley Planning Region,Connecticut,452303.0,86365.0,0
108,Pinal|Arizona,Pinal,Arizona,449219.0,77588.0,0
310,Greater Bridgeport Planning Region|Connecticut,Greater Bridgeport Planning Region,Connecticut,326296.0,87135.0,0


✅ Saved target_expansion_opportunities.csv


PURPOSE:
 Summarize opportunities by state to highlight where expansion potential is highest.


In [90]:

opps = pd.read_csv("target_expansion_opportunities.csv")

state_summary = (
    opps.groupby("State")
    .agg(
        Opportunity_Count=("County_State_Key", "count"),
        Avg_Population=("Population_2023", "mean"),
        Avg_Income=("Median_HH_Income_2023", "mean")
    )
    .reset_index()
    .sort_values("Opportunity_Count", ascending=False)
)

display(state_summary.head(15))
state_summary.to_csv("target_opportunity_summary_by_state.csv", index=False)
print("✅ Saved target_opportunity_summary_by_state.csv")


,State,Opportunity_Count,Avg_Population,Avg_Income
6,Georgia,14,377629.785714,91550.071429
3,Connecticut,8,437814.875000,94024.625000
18,North Carolina,4,156600.500000,72626.750000
21,Pennsylvania,3,130678.000000,74774.000000
24,Texas,3,160959.000000,87762.333333
14,Michigan,2,137957.000000,77763.500000
19,Ohio,2,113083.500000,72972.000000
16,New Jersey,2,137282.500000,126884.500000
11,Louisiana,2,137088.000000,85441.500000
0,Alabama,2,167266.000000,88538.500000


✅ Saved target_opportunity_summary_by_state.csv
